# Testing Misc Methods

## Dataloader with augmentation test

In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import imageio

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import torch.nn as nn

from imgaug import parameters as iap
from imgaug import augmenters as iaa
import imgaug as ia


%matplotlib inline

# Custom includes
from dataloaders.alphapilot import AlphaPilotSegmentation
from dataloaders import utils

class Args():
    input_images_path = 'data/dataset/train/images'
    label_images_path = 'data/dataset/train/labels'

args = Args()
testBatchSize = 16
results_store_dir = 'data/test-aug'
imsize = 512

augs_test = iaa.Sequential([
    # Geometric Augs
    iaa.Resize((imsize, imsize), interpolation='cubic'),
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Rot90((0, 4)),
    
    # Blur and Noise
    iaa.Sometimes(0.15, iaa.OneOf([iaa.GaussianBlur(sigma=(1.5, 2.5), name="gaus-blur"),
                                   iaa.MotionBlur(k=13, angle=[0, 180, 270, 360], direction=[-1.0, 1.0],
                                                 name='motion-blur'),
                                  ])),

    # Color, Contrast, etc
    iaa.Sometimes(0.5, iaa.CoarseDropout(0.05, size_px=(2,4), per_channel=1.0, min_size=2, name='dropout')),
    iaa.SomeOf((0, None), [ iaa.Sometimes(0.5, iaa.GammaContrast((0.5, 1.5), name="contrast")),    
                            iaa.Sometimes(0.5, iaa.Multiply((0.40, 1.60), per_channel=1.0, name="brightness")),
                            iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-30, 30), name="hue-sat")),
                          ]),
])

db_test = AlphaPilotSegmentation(
    input_dir=args.input_images_path, label_dir=args.label_images_path,
    transform=augs_test,
    input_only=["gaus-blur", "motion-blur", "brightness", "contrast",
                "hue-sat", "dropout"]
)
testloader = DataLoader(db_test, batch_size=testBatchSize, shuffle=False, num_workers=4, drop_last=True)

for ii, sample_batched in enumerate(testloader):
    inputs, labels, sample_filename = sample_batched
    print(sample_filename)
    
    labels_colormap = utils.decode_seg_map_sequence(labels.squeeze(1).numpy()).type(torch.FloatTensor)    
    
#     sample = torch.cat((inputs, labels_colormap), 0)
#     img_grid = make_grid(sample, nrow=4, padding=2)

    imgs_per_row = 8
    images = []
    for i in range (0, testBatchSize):
        images.append(inputs[i])
        images.append(labels_colormap[i])
    
    img_grid = make_grid(images, nrow=imgs_per_row, padding=2)
    
    save_image(img_grid, os.path.join(results_store_dir, sample_filename[0] + '-results.png'))


# Inference AlphaPilot Test

In [ ]:
import cv2
from inference_alphapilot import inferenceAlphaPilot
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline


inference = inferenceAlphaPilot(checkpoint_path = 'checkpoint/checkpoint.pth'
                                )
pathInputImage = 'data/dataset/test/images/IMG_0008.JPG'
img =cv2.imread(pathInputImage)
img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

mask = inference.inferenceOnNumpy(img)
print('mask shape:', mask.shape, 'mask dtype:', mask.dtype,'mask max value:', np.amax(mask))

plt.imshow(mask)

